In [1]:
import gym
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from collections import deque

c:\Users\Skygate OPR5\AppData\Local\Programs\Python\Python310\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
env = gym.make('MountainCar-v0')

c:\Users\Skygate OPR5\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\Skygate OPR5\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
env.reset()

array([-0.46767002,  0.        ], dtype=float32)

In [5]:
env.render()

c:\Users\Skygate OPR5\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


None


In [6]:
env.observation_space.shape

(2,)

In [5]:
env.action_space.n

3

In [7]:
env.action_space.sample()

2

In [11]:
class DQN:
    def __init__ (self, env):
        self.env = env
        self.memory = deque(maxlen=2000)
        self.gamma = 0.03
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.005
        self.tau = 0.125
        self.model = self.create_model()
        self.target_model = self.create_model()
    
    def create_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.env.observation_space.shape[0], activation='relu'))
        model.add(Dense(48, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.env.action_space.n))
        model.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=self.learning_rate))
        return model
    
    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.model.predict(state)[0])
    
    def replay(self):
        batch_size = 32
        if len(self.memory) < batch_size:
            return
        
        samples = random.sample(self.memory, batch_size)
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state)[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state, target, epochs=1, verbose=1)
    
    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])
        
    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)
    
    def save_model(self, fn):
        self.model.save(fn)

In [9]:
env = gym.make('MountainCar-v0')
gamma = 0.9
epsilon = 0.95

c:\Users\Skygate OPR5\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\Skygate OPR5\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [12]:
trails = 1000
trail_len = 500
dqn_agnet = DQN(env=env)

In [13]:
current_state = env.reset().reshape(1, 2)
current_state

array([[-0.5545598,  0.       ]], dtype=float32)

In [14]:
dqn_agnet.act(current_state)

0

In [ ]:
steps = []
for trail in range(trails):
    current_state = env.reset().reshape(1, 2)
    for step in range(trail_len):
        print('#', step)
        action = dqn_agnet.act(current_state)
        new_state, reward, done, _ = env.step(action)
        new_state = new_state.reshape(1, 2)
        dqn_agnet.remember(current_state, action, reward, new_state, done)
        dqn_agnet.replay()
        dqn_agnet.target_train()
        
        current_state = new_state
        if done:
            break
    if step >= 199:
        print('Faild')
    else:
        print('success')
        dqn_agnet.save_model('masoud')
        break

# 0
# 1
# 2
# 3
# 4
# 5
# 6
# 7
# 8
# 9
# 10
# 11
# 12
# 13
# 14
# 15
# 16
# 17
# 18
# 19
# 20
# 21
# 22
# 23
# 24
# 25
# 26
# 27
# 28
# 29
# 30
# 31
1/1 [==============================] - 0s 7ms/step - loss: 0.1641
# 32
1/1 [==============================] - 0s 4ms/step - loss: 0.1469
# 33
1/1 [==============================] - 0s 16ms/step - loss: 0.1899
# 34
1/1 [==============================] - 0s 10ms/step - loss: 0.1451


1/1 [==============================] - 0s 4ms/step - loss: 0.1832
# 35
1/1 [==============================] - 0s 4ms/step - loss: 0.1242
# 36
1/1 [==============================] - 0s 5ms/step - loss: 0.1546
# 37
1/1 [==============================] - 0s 4ms/step - loss: 0.1306
# 38
1/1 [==============================] - 0s 17ms/step - loss: 0.2056


1/1 [==============================] - 0s 4ms/step - loss: 0.1688
# 39
1/1 [==============================] - 0s 5ms/step - loss: 0.1011
# 40
1/1 [==============================] - 0s 5ms/step - loss: 0.0963
# 41
1/1 [==============================] - 0s 5ms/step - loss: 0.1023
# 42
1/1 [==============================] - 0s 5ms/step - loss: 0.1258


1/1 [==============================] - 0s 5ms/step - loss: 0.0757
# 43
1/1 [==============================] - 0s 4ms/step - loss: 0.1016
# 44
1/1 [==============================] - 0s 4ms/step - loss: 0.0878
# 45
1/1 [==============================] - 0s 4ms/step - loss: 0.0420
# 46
1/1 [==============================] - 0s 4ms/step - loss: 0.1104


1/1 [==============================] - 0s 10ms/step - loss: 0.0654
# 47
1/1 [==============================] - 0s 19ms/step - loss: 0.0622
# 48
1/1 [==============================] - 0s 5ms/step - loss: 0.0441
# 49
1/1 [==============================] - 0s 4ms/step - loss: 0.0671
# 50
1/1 [==============================] - 0s 5ms/step - loss: 0.0600


...

1/1 [==============================] - 0s 4ms/step - loss: 5.7039e-06
# 198
1/1 [==============================] - 0s 3ms/step - loss: 4.3374e-06
# 199
1/1 [==============================] - 0s 11ms/step - loss: 1.9651e-06
Faild
# 0
1/1 [==============================] - 0s 9ms/step - loss: 4.3839e-07
# 1
1/1 [==============================] - 0s 4ms/step - loss: 2.6351e-06


1/1 [==============================] - 0s 4ms/step - loss: 7.6882e-07
# 2
1/1 [==============================] - 0s 5ms/step - loss: 2.9404e-07
# 3
1/1 [==============================] - 0s 4ms/step - loss: 1.9957e-07
# 4
1/1 [==============================] - 0s 4ms/step - loss: 1.2061e-04


1/1 [==============================] - 0s 5ms/step - loss: 1.2944e-04
# 5
1/1 [==============================] - 0s 4ms/step - loss: 3.5083e-05
# 6
1/1 [==============================] - 0s 6ms/step - loss: 1.7675e-05
# 7
1/1 [==============================] - 0s 5ms/step - loss: 7.3415e-06
# 8
1/1 [==============================] - 0s 5ms/step - loss: 7.6886e-06


1/1 [==============================] - 0s 10ms/step - loss: 9.1879e-07
# 9
1/1 [==============================] - 0s 5ms/step - loss: 3.7555e-07
# 10
1/1 [==============================] - 0s 7ms/step - loss: 2.1582e-06
# 11
1/1 [==============================] - 0s 5ms/step - loss: 4.9750e-07
# 12
1/1 [==============================] - 0s 5ms/step - loss: 2.9324e-06


1/1 [==============================] - 0s 5ms/step - loss: 1.5750e-06
# 13
1/1 [==============================] - 0s 5ms/step - loss: 2.7650e-06
# 14
1/1 [==============================] - 0s 4ms/step - loss: 1.7626e-07
# 15
1/1 [==============================] - 0s 4ms/step - loss: 9.1653e-07
